# Distribuire in un endpoint batch

Si supponga che una clinica sanitaria prenda le misurazioni dei pazienti tutto il giorno, salvando i dettagli per ogni paziente in un file separato. Quindi durante la notte, il modello di stima del diabete può essere usato per elaborare tutti i dati dei pazienti del giorno come batch, generando stime che saranno in attesa il mattino seguente in modo che la clinica possa seguire con i pazienti che sono previsti essere a rischio di diabete. Con Azure Machine Learning è possibile eseguire questa operazione creando un endpoint batch; ed è ciò che si implementerà in questo esercizio.

## Prima di iniziare

È necessaria la versione più recente del pacchetto  **azureml-ai-ml** per eseguire il codice in questo notebook. Eseguire la cella seguente per verificare che sia installata.

> **Nota**:
> Se il pacchetto **azure-ai-ml** non è installato, eseguirlo `pip install azure-ai-ml` per installarlo.

In [ ]:
## Connettersi all'area di lavoro

Con i pacchetti SDK necessari installati, è ora possibile connettersi all'area di lavoro.

Per connettersi a un'area di lavoro, sono necessari parametri di identificatore: ID sottoscrizione, nome del gruppo di risorse e nome dell'area di lavoro. Il nome del gruppo di risorse e il nome dell'area di lavoro sono già compilati. È necessario solo l'ID sottoscrizione per completare il comando.

Per trovare i parametri necessari, fare clic sulla sottoscrizione e sul nome dell'area di lavoro nella parte superiore destra di Studio. Un riquadro verrà aperto a destra.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiare l'ID sottoscrizione e sostituire **YOUR-SUBSCRIPTION-ID** con il valore copiato. </p>

## Registrare il modello

Le distribuzioni batch possono distribuire solo i modelli registrati nell'area di lavoro. Verrà registrato un modello MLflow archiviato nella cartella locale `model` . 

In [ ]:
## Creare un endpoint batch

Un endpoint batch è un endpoint HTTPS che le applicazioni possono chiamare per attivare un processo di assegnazione dei punteggi batch. Un nome dell'endpoint batch deve essere univoco all'interno di un'area di Azure. Si userà la `datetime` funzione per generare un nome univoco in base alla data e all'ora correnti. 

In [ ]:
Per creare un endpoint con la `BatchEndpoint` classe, è necessario specificare il nome e facoltativamente una descrizione. Dopo aver creato un endpoint, si distribuirà un modello nell'endpoint.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANTE Attendere fino a quando l'endpoint non viene creato prima di continuare. Verrà visualizzata una notifica verde nello studio. </p>

## Creare la distribuzione

Una distribuzione è un set di risorse necessarie per ospitare il modello che esegue l'inferenza effettiva. Verrà creata una distribuzione per l'endpoint usando la `BatchDeployment` classe . 

Poiché si distribuisce un modello MLflow, non è necessario uno script di assegnazione dei punteggi o definire l'ambiente. Azure Machine Learning creerà automaticamente tali asset. Il `MLmodel` file nella `model` cartella viene usato per comprendere gli input e gli output previsti sono del modello.

Si distribuirà un modello con i parametri seguenti:

- `name`: nome della distribuzione.
- `description`: Descrizione facoltativa per chiarire ulteriormente cosa rappresenta la distribuzione.
- `endpoint_name`: nome dell'endpoint creato in precedenza, è necessario distribuire il modello.
- `model`: nome del modello registrato.
- `compute`: calcolo da usare quando si richiama il modello distribuito per generare stime.
- `instance_count`: conteggio dei nodi di calcolo da usare per la generazione di stime.
- `max_concurrency_per_instance`: numero massimo di script di assegnazione dei punteggi paralleli eseguiti per ogni nodo di calcolo.
- `mini_batch_size`: numero di file passati per esecuzione dello script di assegnazione dei punteggi.
- `output_action`: ogni nuova stima verrà aggiunta come nuova riga al file di output.
- `output_file_name`: file a cui verranno aggiunte le stime.
- `retry_settings`: le impostazioni per un mini-batch hanno esito negativo.
- `logging_level`: livello di verbosità del log. I valori consentiti sono `warning`, `info`e `debug`. 

L'esecuzione della cella seguente configura e crea la distribuzione.

In [ ]:
<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANTE Attendere il completamento della distribuzione prima di continuare. Verrà visualizzata una notifica verde nello studio. </p>

È possibile distribuire più modelli in un endpoint. È possibile impostare la distribuzione predefinita per specificare quale modello deve essere usato per impostazione predefinita quando si chiama un endpoint batch.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANTE Attendere fino a quando la distribuzione predefinita non è impostata prima di continuare. Verrà visualizzata una notifica verde nello studio. </p>

## Preparare i dati per le stime batch

`data` Nella cartella sono disponibili file CSV con dati non etichettati. Si creerà un asset di dati che punta ai file nella `data` cartella, che verrà usato come input per il processo batch.

In [ ]:
## Inviare il processo

Dopo aver distribuito un modello in un endpoint batch e avere un asset dati senza etichetta, è possibile richiamare l'endpoint per generare stime sui dati non etichettati.

Prima di tutto, si definirà l'input facendo riferimento all'asset di dati registrato. Verrà quindi richiamato l'endpoint, che invierà un processo della pipeline. È possibile usare l'URL del processo per monitorarlo in Studio. Il processo conterrà un processo figlio che rappresenta l'esecuzione dello script di assegnazione dei punteggi (generato) per ottenere le stime.

## Ottenere i risultati

Al termine del processo della pipeline che richiama l'endpoint batch, è possibile visualizzare i risultati. Tutte le stime vengono raccolte nel `predictions.csv` file archiviato nell'archivio dati predefinito. È possibile scaricare il file e visualizzare i dati eseguendo le celle seguenti. 

In [ ]:
from azure.ai.ml.entities import BatchEndpoint

# create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for classifying diabetes in patients",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the endpoint is created before continuing! A green notification should appear in the studio. </p>

## Create the deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `BatchDeployment` class. 

Since you're deploying an MLflow model, you don't need a scoring script or define the environment. Azure Machine Learning will automatically create those assets for you. The `MLmodel` file in the `model` folder is used to understand what the expected inputs and outputs are of the model.

You'll deploy a model with the following parameters:

- `name`: Name of the deployment.
- `description`: Optional description to further clarify what the deployment represents.
- `endpoint_name`: Name of the previously created endpoint the model should be deployed to.
- `model`: Name of the registered model.
- `compute`: Compute to be used when invoking the deployed model to generate predictions.
- `instance_count`: Count of compute nodes to use for generating predictions.
- `max_concurrency_per_instance`: Maximum number of parallel scoring script runs per compute node.
- `mini_batch_size`: Number of files passed per scoring script run.
- `output_action`: Each new prediction will be appended as a new row to the output file.
- `output_file_name`: File to which predictions will be appended.
- `retry_settings`: Settings for a mini-batch fails.
- `logging_level`: The log verbosity level. Allowed values are `warning`, `info`, and `debug`. 

Running the following cell will configure and create the deployment.

In [ ]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

deployment = BatchDeployment(
    name="classifier-diabetes-mlflow",
    description="A diabetes classifier",
    endpoint_name=endpoint.name,
    model=model,
    compute="aml-cluster",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)
ml_client.batch_deployments.begin_create_or_update(deployment)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio. </p>

You can deploy multiple models to an endpoint. You can set the default deployment to specify which model should be used by default when calling a batch endpoint.

In [ ]:
endpoint.defaults = {}

endpoint.defaults["deployment_name"] = deployment.name

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the default deployment is set before continuing! A green notification should appear in the studio. </p>

## Prepare the data for batch predictions

In the `data` folder you'll find CSV files with unlabeled data. You'll create a data asset that points to the files in the `data` folder, which you'll use as input for the batch job.

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "./data"
dataset_name = "patient-data-unlabeled"

patient_dataset_unlabeled = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="An unlabeled dataset for diabetes classification",
    name=dataset_name,
)
ml_client.data.create_or_update(patient_dataset_unlabeled)

In [ ]:
patient_dataset_unlabeled = ml_client.data.get(
    name="patient-data-unlabeled", label="latest"
)

## Submit the job

Now that you have deployed a model to a batch endpoint, and have an unlabeled data asset, you're ready to invoke the endpoint to generate predictions on the unlabeled data.

First, you'll define the input by referring to the registered data asset. Then, you'll invoke the endpoint, which will submit a pipeline job. You can use the job URL to monitor it in the Studio. The job will contain a child job that represents the running of the (generated) scoring script to get the predictions.

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

input = Input(type=AssetTypes.URI_FOLDER, path=patient_dataset_unlabeled.id)

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
    input=input)

ml_client.jobs.get(job.name)

## Get the results

When the pipeline job that invokes the batch endpoint is completed, you can view the results. All predictions are collected in the `predictions.csv` file that is stored in the default datastore. You can download the file and visualize the data by running the following cells. 

In [ ]:
ml_client.jobs.download(name=job.name, download_path=".", output_name="score")

In [ ]:
with open("predictions.csv", "r") as f:
    data = f.read()

In [ ]:
from ast import literal_eval
import pandas as pd

score = pd.DataFrame(
    literal_eval(data.replace("\n", ",")), columns=["file", "prediction"]
)
score